In [ ]:
from langgraph.graph import StateGraph , START , END
from typing import TypedDict , Literal , Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from pydantic import BaseModel

SyntaxError: invalid syntax (3134065872.py, line 1)

In [ ]:
load_dotenv()

google_apikey  = os.getenv("GEMINI_API_KEY")

generator_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)


evaluation_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)


optimizer_llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-pro",
    api_key = google_apikey
)

In [ ]:
class TopicState(TypedDict):
    topic:str
    generate_tweet:str
    evaluation : Literal['Approved' , "needs_improvement"]
    feedback :str
    iteration : int
    max_iteration : int





